# Football Transfer Data Project
##### by David VanHeeswijk

The purpose of this notebook is to explore Data sets found from [Kaggle- European Football Transfers Dataset](https://www.kaggle.com/giovannibeli/european-football-transfers-database).  In this dataset, we find data ranging across many areas of European football, including:
* Player Stats
* Player Information
* Club Records and Stats
* Transfers from season to season across Europe's football leagues
* League Basic information
* Coaches and Stadia
* National Team stats for players
* etc.

We would like to answer the following questions:
1. What are the best indicators for predicting market value and transfer fees?
2. Which nationality produces the best players for value across the entire European football system?
3. What player positions produce the most *bang for your buck*?
4. What is the ideal age to purchase/sell a player?

In this notebook, we will primarily look to wrangle in the data, merging several of the csv files into one unified data set that has a limited number of features from which we can create a model. We start by importing our libraries.

In [689]:
import pandas as pd
import numpy as np

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

from pathlib import Path

import datetime

In [690]:
# Loading in csv files into DataFrames to explore
# First, we will load our data sets containing transfer data

transfers= pd.read_csv('transfers.csv', delimiter=';')

In [691]:
transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111602 entries, 0 to 111601
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   111602 non-null  int64  
 1   player_id            111602 non-null  int64  
 2   player_name          111602 non-null  object 
 3   season               111602 non-null  int64  
 4   date                 111530 non-null  object 
 5   from_club_id         111602 non-null  int64  
 6   from_club_name       111602 non-null  object 
 7   to_club_id           111602 non-null  int64  
 8   to_club_name         111602 non-null  object 
 9   market_value         72589 non-null   float64
 10  fee                  42799 non-null   float64
 11  from_coach_name      38687 non-null   object 
 12  to_coach_name        38690 non-null   object 
 13  from_sport_dir_name  17854 non-null   object 
 14  to_sport_dir_name    18226 non-null   object 
 15  contract_was_till

We want to make a list of features that will be used to create our model for predicting fees. We will narrow it down the these 10:

* season
* position
* nationality
* league
* goals + assists
* total minutes played
* height
* dob
* club position in league
* market value

In [692]:
transfers.columns

Index(['id', 'player_id', 'player_name', 'season', 'date', 'from_club_id',
       'from_club_name', 'to_club_id', 'to_club_name', 'market_value', 'fee',
       'from_coach_name', 'to_coach_name', 'from_sport_dir_name',
       'to_sport_dir_name', 'contract_was_till', 'is_loan', 'is_end_of_loan',
       'is_future_transfer'],
      dtype='object')

In [739]:
new_transfers = transfers[['player_id','player_name', 'season','market_value', 'fee', 'from_club_id', 'from_club_name', 'to_club_id','to_club_name', 'is_loan', 'is_end_of_loan']]
new_transfers['market_value'].fillna(new_transfers['fee'], inplace=True)

new_transfers.head()

C:\Users\david\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name,is_loan,is_end_of_loan
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury,0,0
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE,0,0
2,1,Jermaine Beckford,2014,750000.0,NaN,391,Preston NE,289,Bolton,0,1
3,1,Jermaine Beckford,2014,1200000.0,NaN,289,Bolton,391,Preston NE,1,0
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton,0,0


In [740]:
new_transfers = new_transfers[new_transfers['to_club_name'] != 'Retired']

new_transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108090 entries, 0 to 111601
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   player_id       108090 non-null  int64  
 1   player_name     108090 non-null  object 
 2   season          108090 non-null  int64  
 3   market_value    82487 non-null   float64
 4   fee             42783 non-null   float64
 5   from_club_id    108090 non-null  int64  
 6   from_club_name  108090 non-null  object 
 7   to_club_id      108090 non-null  int64  
 8   to_club_name    108090 non-null  object 
 9   is_loan         108090 non-null  int64  
 10  is_end_of_loan  108090 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 9.9+ MB


In [741]:
new_transfers.loc[(new_transfers['fee'].isnull())&(new_transfers['is_loan'] + new_transfers['is_end_of_loan'] > 0),'fee'] = 0

new_transfers.head()

,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name,is_loan,is_end_of_loan
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury,0,0
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE,0,0
2,1,Jermaine Beckford,2014,750000.0,0.0,391,Preston NE,289,Bolton,0,1
3,1,Jermaine Beckford,2014,1200000.0,0.0,289,Bolton,391,Preston NE,1,0
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton,0,0


In [742]:
new_transfers['is_loan'] = new_transfers['is_loan'] + new_transfers['is_end_of_loan']
new_transfers.drop('is_end_of_loan', axis=1, inplace=True)
new_transfers.head()

,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name,is_loan
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury,0
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE,0
2,1,Jermaine Beckford,2014,750000.0,0.0,391,Preston NE,289,Bolton,1
3,1,Jermaine Beckford,2014,1200000.0,0.0,289,Bolton,391,Preston NE,1
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton,0


In [743]:
new_transfers['free_transfer'] = True

for i in new_transfers.index:
    if(new_transfers.loc[i,'fee'] > 0)or(new_transfers.loc[i,'is_loan'] == 1):
        new_transfers.loc[i,'free_transfer'] = False
        
new_transfers.head()

,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name,is_loan,free_transfer
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury,0,True
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE,0,True
2,1,Jermaine Beckford,2014,750000.0,0.0,391,Preston NE,289,Bolton,1,False
3,1,Jermaine Beckford,2014,1200000.0,0.0,289,Bolton,391,Preston NE,1,False
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton,0,True


In [744]:
new_transfers.drop('is_loan',axis=1,inplace=True)

In [745]:
new_transfers.head()

,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name,free_transfer
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury,True
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE,True
2,1,Jermaine Beckford,2014,750000.0,0.0,391,Preston NE,289,Bolton,False
3,1,Jermaine Beckford,2014,1200000.0,0.0,289,Bolton,391,Preston NE,False
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton,True


In [746]:
new_transfers.drop('free_transfer',axis=1,inplace=True)

While we are missing some values in the 'fee' and 'market_value' columns, we have essentially gathered the data that we need from the transfers df. Now we will pick up the features from our player stats and dict data sets and combine the three.

In [701]:
player_stats = pd.read_csv('stats_of_players.csv', delimiter=';')
player_dict = pd.read_csv('dict_players.csv', delimiter=';')

player_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231379 entries, 0 to 231378
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                231379 non-null  int64  
 1   player_id         231379 non-null  int64  
 2   player_name       231379 non-null  object 
 3   season            231379 non-null  int64  
 4   league_id         231379 non-null  int64  
 5   league_name       231379 non-null  object 
 6   club_id           231379 non-null  int64  
 7   club_name         231379 non-null  object 
 8   apps              231379 non-null  int64  
 9   points_per_match  225422 non-null  float64
 10  goals             166957 non-null  float64
 11  assists           159543 non-null  float64
 12  conceded_goals    118416 non-null  float64
 13  clean_sheets      117263 non-null  float64
 14  yellow_card       190724 non-null  float64
 15  two_yellow_cards  117566 non-null  float64
 16  red_card          11

In [702]:
player_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11382 entries, 0 to 11381
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           11382 non-null  int64  
 1   name                         11382 non-null  object 
 2   original_name                6456 non-null   object 
 3   club_id                      11382 non-null  int64  
 4   club_name                    11382 non-null  object 
 5   position_main                11362 non-null  object 
 6   other_positions              8043 non-null   object 
 7   nationality_name             11382 non-null  object 
 8   nationality_code             11088 non-null  object 
 9   other_nationality_name       3427 non-null   object 
 10  other_nationality_code       3215 non-null   object 
 11  date_of_birth                11362 non-null  object 
 12  place_of_birth_name          11278 non-null  object 
 13  place_of_birth_c

We will start with the player_stats dataframe, and gather only the columns that we need for our model.

In [703]:
player_stats.columns

Index(['id', 'player_id', 'player_name', 'season', 'league_id', 'league_name',
       'club_id', 'club_name', 'apps', 'points_per_match', 'goals', 'assists',
       'conceded_goals', 'clean_sheets', 'yellow_card', 'two_yellow_cards',
       'red_card', 'minutes_played'],
      dtype='object')

In [705]:
player_stats = player_stats.groupby(['player_id','player_name','season','club_name']).sum()[['goals','assists', 'apps', 'minutes_played']]

player_stats.reset_index().head()

,player_id,player_name,season,club_name,goals,assists,apps,minutes_played
0,1,Jermaine Beckford,2005,Leeds United,0.0,0.0,5,137
1,1,Jermaine Beckford,2006,Carlisle United,9.0,3.0,27,1979
2,1,Jermaine Beckford,2007,Leeds United,20.0,1.0,45,3789
3,1,Jermaine Beckford,2008,Leeds United,34.0,1.0,41,3283
4,1,Jermaine Beckford,2009,Leeds United,30.0,9.0,51,4184


In [706]:
player_stats['goal_contributions'] = player_stats['goals']+player_stats['assists']
player_stats['minutes_per_appearance'] = player_stats['minutes_played']//player_stats['apps']

player_stats.drop(['goals', 'assists', 'apps'],axis=1, inplace=True)

player_stats.head()

minutes_played  \
player_id player_name       season club_name                         
1         Jermaine Beckford 2005   Leeds United                137   
                            2006   Carlisle United            1979   
                            2007   Leeds United               3789   
                            2008   Leeds United               3283   
                            2009   Leeds United               4184   

                                                    goal_contributions  \
player_id player_name       season club_name                             
1         Jermaine Beckford 2005   Leeds United                    0.0   
                            2006   Carlisle United                12.0   
                            2007   Leeds United                   21.0   
                            2008   Leeds United                   35.0   
                            2009   Leeds United                   39.0   

                                                    minutes_per_appearance  
player_id player_name       season club_name                                
1         Jermaine Beckford 2005   Leeds United                         27  
                            2006   Carlisle United                      73  
                            2007   Leeds United                         84  
                            2008   Leeds United                         80  
                            2009   Leeds United                         82

In [707]:
player_stats.reset_index(inplace=True)

In [708]:
player_stats.head()

,player_id,player_name,season,club_name,minutes_played,goal_contributions,minutes_per_appearance
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73
2,1,Jermaine Beckford,2007,Leeds United,3789,21.0,84
3,1,Jermaine Beckford,2008,Leeds United,3283,35.0,80
4,1,Jermaine Beckford,2009,Leeds United,4184,39.0,82


Now that we have gathered the relevant columns that we will need for our analysis from the stats dataframe, we pull in the player info from the player_dict dataframe and merge the two together.

In [709]:
player_dict.columns

Index(['id', 'name', 'original_name', 'club_id', 'club_name', 'position_main',
       'other_positions', 'nationality_name', 'nationality_code',
       'other_nationality_name', 'other_nationality_code', 'date_of_birth',
       'place_of_birth_name', 'place_of_birth_country_name',
       'place_of_birth_country_code', 'foot', 'height', 'player_agent',
       'joined', 'contract_until', 'outfiter', 'last_extention',
       'contract_options', 'current_market_value'],
      dtype='object')

In [711]:
player_info = player_dict[['id','name','position_main', 'nationality_name','nationality_code', 'date_of_birth', 'height']]

player_info.head()

,id,name,position_main,nationality_name,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,Centre-Forward,Jamaica,JAM,1983-12-09,188.0
1,2,Harry Charsley,Central Midfield,Ireland,IRL,1996-11-01,NaN
2,3,Mark Davies,Central Midfield,England,GBR,1988-02-18,180.0
3,4,Alex McQuade,Centre-Back,England,GBR,1992-11-07,NaN
4,5,Przemyslaw Kazimierczak,Goalkeeper,Poland,POL,1988-05-05,191.0


In [712]:
player_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11382 entries, 0 to 11381
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                11382 non-null  int64  
 1   name              11382 non-null  object 
 2   position_main     11362 non-null  object 
 3   nationality_name  11382 non-null  object 
 4   nationality_code  11088 non-null  object 
 5   date_of_birth     11362 non-null  object 
 6   height            11046 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 622.6+ KB


It seems we are missing a few nationalities and dobs, as well as a few positions. We can leave the null position data for now and focus more on the nationalities missing, as well as dates of birth.

In [713]:
player_info[player_info['nationality_code'].isnull()]

,id,name,position_main,nationality_name,nationality_code,date_of_birth,height
56,57,Sead Kolasinac,Left-Back,Bosnia-Herzegovina,NaN,1993-06-20,183.0
100,101,Wilfried Bony,Centre-Forward,Cote d'Ivoire,NaN,1988-12-10,181.0
130,131,Franck Kessié,Central Midfield,Cote d'Ivoire,NaN,1996-12-19,183.0
131,132,Didier Drogba,Centre-Forward,Cote d'Ivoire,NaN,1978-03-11,189.0
138,139,Zvjezdan Misimovic,Attacking Midfield,Bosnia-Herzegovina,NaN,1982-06-05,179.0
...,...,...,...,...,...,...,...
11321,11322,Clarck Nsikulu,Left Winger,DR Congo,NaN,1992-07-10,180.0
11325,11326,Yohan Boli,Centre-Forward,Cote d'Ivoire,NaN,1993-11-17,181.0
11345,11346,Wilfred Moke,Defensive Midfield,DR Congo,NaN,1988-02-12,183.0
11356,11357,Elie Kroupi,Centre-Forward,Cote d'Ivoire,NaN,1979-10-18,175.0


In [714]:
player_info[player_info['nationality_code'].isnull()]['nationality_name'].unique()

array(['Bosnia-Herzegovina', "Cote d'Ivoire", 'DR Congo', 'Curacao',
       'Tahiti', 'Cape Verde', 'Kosovo', 'Korea, North', 'Palästina',
       'Chinese Taipei (Taiwan)'], dtype=object)

We see that there are a few countries that pop up with no nationality code. After a quick search of FIFA country codes, we found the missing info and will fill it in now.

In [715]:
country_codes = [['Bosnia-Herzegovina', 'BIH'],["Cote d'Ivoire", 'CIV'],['DR Congo', 'CGO'],
                 ['Curacao', 'CUW'], ['Tahiti', 'TAH'], ['Cape Verde', 'CPV'],
                 ['Kosovo', 'KVX'], ['Korea, North', 'PRK'], ['Palästina', 'PLE'], ['Chinese Taipei (Taiwan)', 'TPE']]

for country in country_codes:
    for ind in player_info.index:
        if country[0] == player_info.loc[ind,'nationality_name']:
            player_info.loc[ind,'nationality_code'] = country[1]
            
player_info.info()

C:\Users\david\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11382 entries, 0 to 11381
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                11382 non-null  int64  
 1   name              11382 non-null  object 
 2   position_main     11362 non-null  object 
 3   nationality_name  11382 non-null  object 
 4   nationality_code  11382 non-null  object 
 5   date_of_birth     11362 non-null  object 
 6   height            11046 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 622.6+ KB


In [716]:
player_info[player_info['date_of_birth'].isnull()]

,id,name,position_main,nationality_name,nationality_code,date_of_birth,height
20,21,Karim Matmour,Right Winger,Algeria,DZA,NaN,181.0
3408,3409,Mineiro,Defensive Midfield,Brazil,BRA,NaN,169.0
3947,3948,David Odonkor,Right Winger,Germany,DEU,NaN,172.0
5485,5486,Carsten Ramelow,Defensive Midfield,Germany,DEU,NaN,186.0
7209,7210,Faruk Namdar,Attacking Midfield,Turkey,TUR,NaN,184.0
7549,7550,Markus Bollmann,Centre-Back,Germany,DEU,NaN,190.0
7841,7842,Markus Kurth,Centre-Forward,Germany,DEU,NaN,180.0
7922,7923,Philipp Bönig,Left-Back,Germany,DEU,NaN,175.0
8378,8379,Daniel Halfar,Attacking Midfield,Germany,DEU,NaN,173.0
8631,8632,Moses Sichone,Centre-Back,Zambia,ZMB,NaN,187.0


After doing some quick searches, we realize that these players are all retired, so rather than filling in the information now, we will wait until after we have merged the player info to the stats and transfer data frames before deciding if these need to be fixed.

As for height, we will use a mean function grouped around position to fill in the missing data, as there are too many mising entries to fill in via a search online.

In [717]:
player_info[player_info['height'].isnull()][['name','position_main']].head(10)

,name,position_main
1,Harry Charsley,Central Midfield
3,Alex McQuade,Centre-Back
76,Marcelo Bordon,Centre-Back
317,Michael Ballack,Central Midfield
335,Ewerthon,Centre-Forward
388,Fernando Morientes,Centre-Forward
468,Cris,Centre-Back
595,Martin Petrov,Left Winger
773,Maniche,Central Midfield
779,Tomás Ujfalusi,Centre-Back


In [718]:
height_means = player_info.groupby('position_main')['height'].mean().reset_index()

height_means = pd.DataFrame(height_means)

A note can be made that we can simplify the positions listed, since many of the positions are similar, such as Centre-Back and Sweeper or Right Winger and Right Midfield. We can combine common positions to help make our analysis more realistic, since positions names are relative to formations, which shouldn't matter in regards to the market value.

In [719]:
for ind in player_info.index:
    if np.isnan(player_info.loc[ind,'height']):
        for height in height_means.index:
            if height_means.loc[height,'position_main'] == player_info.loc[ind,'position_main']:
                player_info.loc[ind,'height'] = height_means.loc[height,'height']
            
player_info.info()

C:\Users\david\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11382 entries, 0 to 11381
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                11382 non-null  int64  
 1   name              11382 non-null  object 
 2   position_main     11362 non-null  object 
 3   nationality_name  11382 non-null  object 
 4   nationality_code  11382 non-null  object 
 5   date_of_birth     11362 non-null  object 
 6   height            11381 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 622.6+ KB


In [720]:
# Removing the only player in our database that has no position or height.
player_info = player_info[player_info['name'] != 'Müslim Can']
player_info.head()

,id,name,position_main,nationality_name,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,Centre-Forward,Jamaica,JAM,1983-12-09,188.000000
1,2,Harry Charsley,Central Midfield,Ireland,IRL,1996-11-01,179.777417
2,3,Mark Davies,Central Midfield,England,GBR,1988-02-18,180.000000
3,4,Alex McQuade,Centre-Back,England,GBR,1992-11-07,186.662449
4,5,Przemyslaw Kazimierczak,Goalkeeper,Poland,POL,1988-05-05,191.000000


In [721]:
player_info['position_main'].unique()

array(['Centre-Forward', 'Central Midfield', 'Centre-Back', 'Goalkeeper',
       'Right Winger', 'Left Winger', 'Second Striker',
       'Defensive Midfield', 'Attacking Midfield', 'Right Midfield',
       'Right-Back', 'Left-Back', 'Left Midfield', nan, 'Sweeper'],
      dtype=object)

In [722]:
positions = {'Centre-Forward':'S', 'Central Midfield': 'CM', 'Centre-Back':'CB', 'Goalkeeper':'GK',
       'Right Winger':'RM', 'Left Winger':'LM', 'Second Striker':'S',
       'Defensive Midfield':'CDM', 'Attacking Midfield':'CAM', 'Right Midfield':'RM',
       'Right-Back':'RB', 'Left-Back':'LB', 'Left Midfield':'LM', 'Sweeper':'CB'}

player_info.replace({'position_main':positions}, inplace=True)
player_info.head()

,id,name,position_main,nationality_name,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,S,Jamaica,JAM,1983-12-09,188.000000
1,2,Harry Charsley,CM,Ireland,IRL,1996-11-01,179.777417
2,3,Mark Davies,CM,England,GBR,1988-02-18,180.000000
3,4,Alex McQuade,CB,England,GBR,1992-11-07,186.662449
4,5,Przemyslaw Kazimierczak,GK,Poland,POL,1988-05-05,191.000000


In [723]:
player_info.rename(columns={'position_main':'position', 'name':'player_name'},inplace=True)
player_info.head()

,id,player_name,position,nationality_name,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,S,Jamaica,JAM,1983-12-09,188.000000
1,2,Harry Charsley,CM,Ireland,IRL,1996-11-01,179.777417
2,3,Mark Davies,CM,England,GBR,1988-02-18,180.000000
3,4,Alex McQuade,CB,England,GBR,1992-11-07,186.662449
4,5,Przemyslaw Kazimierczak,GK,Poland,POL,1988-05-05,191.000000


Now, we will merge the two player specific columns together, using player name as the point to merge.

In [731]:
player_info = player_info[['id','player_name','position','nationality_code','date_of_birth','height']]
player_df = pd.merge(player_stats,player_info, how='left', left_on='player_id', right_on='id')

player_df.head()

,player_id,player_name_x,season,club_name,minutes_played,goal_contributions,minutes_per_appearance,id,player_name_y,position,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27,1.0,Jermaine Beckford,S,JAM,1983-12-09,188.0
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,1.0,Jermaine Beckford,S,JAM,1983-12-09,188.0
2,1,Jermaine Beckford,2007,Leeds United,3789,21.0,84,1.0,Jermaine Beckford,S,JAM,1983-12-09,188.0
3,1,Jermaine Beckford,2008,Leeds United,3283,35.0,80,1.0,Jermaine Beckford,S,JAM,1983-12-09,188.0
4,1,Jermaine Beckford,2009,Leeds United,4184,39.0,82,1.0,Jermaine Beckford,S,JAM,1983-12-09,188.0


In [733]:
player_df.drop(columns=['id','player_name_y'], inplace=True)
player_df.rename(columns={'player_name_x':'player_name'}, inplace=True)
player_df.columns

Index(['player_id', 'player_name', 'season', 'club_name', 'minutes_played',
       'goal_contributions', 'minutes_per_appearance', 'position',
       'nationality_code', 'date_of_birth', 'height'],
      dtype='object')

In [734]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139504 entries, 0 to 139503
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   player_id               139504 non-null  int64  
 1   player_name             139504 non-null  object 
 2   season                  139504 non-null  int64  
 3   club_name               139504 non-null  object 
 4   minutes_played          139504 non-null  int64  
 5   goal_contributions      139504 non-null  float64
 6   minutes_per_appearance  139504 non-null  int64  
 7   position                139324 non-null  object 
 8   nationality_code        139491 non-null  object 
 9   date_of_birth           139185 non-null  object 
 10  height                  139491 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 12.8+ MB


In [735]:
player_df = player_df[player_df['date_of_birth'].notnull()]
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139185 entries, 0 to 139503
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   player_id               139185 non-null  int64  
 1   player_name             139185 non-null  object 
 2   season                  139185 non-null  int64  
 3   club_name               139185 non-null  object 
 4   minutes_played          139185 non-null  int64  
 5   goal_contributions      139185 non-null  float64
 6   minutes_per_appearance  139185 non-null  int64  
 7   position                139018 non-null  object 
 8   nationality_code        139185 non-null  object 
 9   date_of_birth           139185 non-null  object 
 10  height                  139185 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 12.7+ MB


In [736]:
player_df[player_df['position'].isnull()]['player_name'].unique()

array(['Hasan Yurt', 'Aleksandar Yordanov Aleksandrov', 'Devran Ayhan',
       'Lubos Pecka', 'Cristiano', 'Serkan Bensol', 'Gastón Curbelo',
       'Pini Balili', 'Laurentiu Rosu', 'Goran Stavrevski',
       'André Paulo Pinto', 'Abdelmajid Oulmers', 'Ramazan Tunc',
       'Emra Tahirovic', 'Cyrille Watier', "Alain N'Kong", 'Mithat Yavas',
       'Fransergio', 'Can Cumhur Bozaci'], dtype=object)

With only 19 players info missing for position, we can just remove these entries and get a dataframe with no null values.

In [737]:
player_df = player_df[player_df['position'].notnull()]
player_df.head(10)

,player_id,player_name,season,club_name,minutes_played,goal_contributions,minutes_per_appearance,position,nationality_code,date_of_birth,height
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27,S,JAM,1983-12-09,188.0
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0
2,1,Jermaine Beckford,2007,Leeds United,3789,21.0,84,S,JAM,1983-12-09,188.0
3,1,Jermaine Beckford,2008,Leeds United,3283,35.0,80,S,JAM,1983-12-09,188.0
4,1,Jermaine Beckford,2009,Leeds United,4184,39.0,82,S,JAM,1983-12-09,188.0
5,1,Jermaine Beckford,2010,Everton FC,1794,17.0,44,S,JAM,1983-12-09,188.0
6,1,Jermaine Beckford,2011,Everton FC,68,0.0,34,S,JAM,1983-12-09,188.0
7,1,Jermaine Beckford,2011,Leicester City,2955,20.0,67,S,JAM,1983-12-09,188.0
8,1,Jermaine Beckford,2012,Leicester City,1663,10.0,61,S,JAM,1983-12-09,188.0
9,1,Jermaine Beckford,2013,Bolton Wanderers,2065,10.0,59,S,JAM,1983-12-09,188.0


Now, we refer back to our transfer data set and combine the columns we want to include.

In [747]:
new_transfers.head()

,player_id,player_name,season,market_value,fee,from_club_id,from_club_name,to_club_id,to_club_name
0,1,Jermaine Beckford,2017,500000.0,0.0,391,Preston NE,392,Bury
1,1,Jermaine Beckford,2015,750000.0,0.0,289,Bolton,391,Preston NE
2,1,Jermaine Beckford,2014,750000.0,0.0,391,Preston NE,289,Bolton
3,1,Jermaine Beckford,2014,1200000.0,0.0,289,Bolton,391,Preston NE
4,1,Jermaine Beckford,2013,1500000.0,NaN,271,Leicester,289,Bolton


In [748]:
df = pd.merge(player_df, new_transfers, how='left', on=['player_id','player_name','season'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170839 entries, 0 to 170838
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   player_id               170839 non-null  int64  
 1   player_name             170839 non-null  object 
 2   season                  170839 non-null  int64  
 3   club_name               170839 non-null  object 
 4   minutes_played          170839 non-null  int64  
 5   goal_contributions      170839 non-null  float64
 6   minutes_per_appearance  170839 non-null  int64  
 7   position                170839 non-null  object 
 8   nationality_code        170839 non-null  object 
 9   date_of_birth           170839 non-null  object 
 10  height                  170839 non-null  float64
 11  market_value            88515 non-null   float64
 12  fee                     80653 non-null   float64
 13  from_club_id            103393 non-null  float64
 14  from_club_name      

In [749]:
df.drop(['from_club_id','from_club_name','to_club_id','to_club_name'],axis=1, inplace=True)

df.head()

,player_id,player_name,season,club_name,minutes_played,goal_contributions,minutes_per_appearance,position,nationality_code,date_of_birth,height,market_value,fee
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27,S,JAM,1983-12-09,188.0,0.0,0.0
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0
2,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0
3,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0
4,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0


The last piece of the dataframe we would like to include is the league that the player plays in. For this information, we will need to access the clubs_in_leagues.csv file as well as the dict_league.csv file.

In [750]:
clubs_leagues = pd.read_csv('clubs_in_leagues.csv', delimiter=';')

clubs_leagues.head()

,Unnamed: 0,id,club_id,club_name,league_id,season,matches_played,matches_overall,wins,draws,loses,goals_scored,goals_cons,goals_diff,points,place,qualified_to,is_champion,is_cup_winner,is_promoted
0,0,1,1,Bayern Munich,4,1999,34,34,22,7,5,73,28,45,73,1,CL,1.0,1.0,0.0
1,1,2,2,Bay. Leverkusen,4,1999,34,34,21,10,3,74,36,38,73,2,CL,0.0,0.0,0.0
2,2,3,3,Hamburger SV,4,1999,34,34,16,11,7,63,39,24,59,3,CL Quals,0.0,0.0,0.0
3,3,4,4,1860 Munich,4,1999,34,34,14,11,9,55,48,7,53,4,CL Quals,0.0,0.0,0.0
4,4,5,5,1.FC K'lautern,4,1999,34,34,15,5,14,54,59,-5,50,5,EL Quals,0.0,0.0,0.0


In [751]:
clubs_leagues.columns

Index(['Unnamed: 0', 'id', 'club_id', 'club_name', 'league_id', 'season',
       'matches_played', 'matches_overall', 'wins', 'draws', 'loses',
       'goals_scored', 'goals_cons', 'goals_diff', 'points', 'place',
       'qualified_to', 'is_champion', 'is_cup_winner', 'is_promoted'],
      dtype='object')

In [752]:
clubs_leagues = clubs_leagues[['club_id', 'club_name', 'league_id']]
clubs_leagues.groupby('club_id').head()

,club_id,club_name,league_id
0,1,Bayern Munich,4
1,2,Bay. Leverkusen,4
2,3,Hamburger SV,4
3,4,1860 Munich,4
4,5,1.FC K'lautern,4
...,...,...,...
3556,389,KV Oostende,9
3559,390,Mouscron,9
3560,386,KAS Eupen,9
3571,390,Mouscron,9


In [753]:
leagues = pd.read_csv('dict_leagues.csv', delimiter=';')

leagues.head()

,id,name,country,country_id,num,evaluation,group
0,1,Premier League,England,GBR,1,8.270000e+09,1
1,2,LaLiga,Spain,ESP,1,5.530000e+09,1
2,3,Serie A,Italy,ITA,1,4.700000e+09,2
3,4,Bundesliga,Germany,DEU,1,4.290000e+09,2
4,5,Ligue 1,France,FRA,1,3.330000e+09,2


In [754]:
leagues = leagues[['id','name']]

leagues.head()

,id,name
0,1,Premier League
1,2,LaLiga
2,3,Serie A
3,4,Bundesliga
4,5,Ligue 1


In [755]:
clubs = pd.merge(clubs_leagues, leagues, how='left',left_on='league_id',right_on='id')
clubs.rename(columns={'name':'league'},inplace=True)
clubs.head()

,club_id,club_name,league_id,id,league
0,1,Bayern Munich,4,4,Bundesliga
1,2,Bay. Leverkusen,4,4,Bundesliga
2,3,Hamburger SV,4,4,Bundesliga
3,4,1860 Munich,4,4,Bundesliga
4,5,1.FC K'lautern,4,4,Bundesliga


In [756]:
df_final = pd.merge(df,clubs[['club_name','league']], how='left', on='club_name')

df_final.head()

,player_id,player_name,season,club_name,minutes_played,goal_contributions,minutes_per_appearance,position,nationality_code,date_of_birth,height,market_value,fee,league
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27,S,JAM,1983-12-09,188.0,0.0,0.0,NaN
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,NaN
2,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,NaN
3,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,NaN
4,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,NaN


In [757]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698950 entries, 0 to 698949
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   player_id               698950 non-null  int64  
 1   player_name             698950 non-null  object 
 2   season                  698950 non-null  int64  
 3   club_name               698950 non-null  object 
 4   minutes_played          698950 non-null  int64  
 5   goal_contributions      698950 non-null  float64
 6   minutes_per_appearance  698950 non-null  int64  
 7   position                698950 non-null  object 
 8   nationality_code        698950 non-null  object 
 9   date_of_birth           698950 non-null  object 
 10  height                  698950 non-null  float64
 11  market_value            330638 non-null  float64
 12  fee                     307717 non-null  float64
 13  league                  572216 non-null  object 
dtypes: float64(4), int64

In [758]:
df_final['league'].fillna('Other League',inplace=True)

df_final.head()

,player_id,player_name,season,club_name,minutes_played,goal_contributions,minutes_per_appearance,position,nationality_code,date_of_birth,height,market_value,fee,league
0,1,Jermaine Beckford,2005,Leeds United,137,0.0,27,S,JAM,1983-12-09,188.0,0.0,0.0,Other League
1,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,Other League
2,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,Other League
3,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,Other League
4,1,Jermaine Beckford,2006,Carlisle United,1979,12.0,73,S,JAM,1983-12-09,188.0,100000.0,0.0,Other League


And there we have it! A data set that contains all the relevant data, with only null values for market value and fee in any spots. We will now save this dataframe to a csv file and explore creating models in the next Jupyter notebook.

In [759]:
df_final.to_csv('football_data.csv',index=False)